In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
from keras.models import Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, confusion_matrix, auc
from sklearn.preprocessing import StandardScaler

# Load dataset from CSV
dataFileName = '/kaggle/input/concatenated-pca/concatenated_data_synthethic_PCA.csv'  # Adjust path as necessary

try:
    raw_dataset = np.genfromtxt(dataFileName, delimiter=',')
    dataset = raw_dataset[:, :]  # Adjust indexing based on your CSV structure
    print(dataset.shape)
except Exception as e:
    print(f"Failed to load or parse the CSV file. Error: {e}")
    exit()

# Split dataset into training and validation sets
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Split features and labels
metadata = train_dataset.shape[1] - 4
x_train = train_dataset[:, :metadata]
x_val = val_dataset[:, :metadata]
y_train = train_dataset[:, metadata:].astype(int)
y_val = val_dataset[:, metadata:].astype(int)

# Normalize features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

# Reshape data to match the input shape of the model (batch size, height, width, channels)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1, 1)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1, 1)

# Define a modified AlexNet model with adjusted pooling size
def alexnet_model(input_shape, num_classes=4):
    inputs = Input(shape=input_shape)
    x = Conv2D(96, (3, 3), strides=(2, 2), activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2, 1))(x)  # Adjusted pooling size
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 1))(x)  # Adjusted pooling size
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(num_classes, activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=x)

# Model setup
input_shape = x_train.shape[1:]
model = alexnet_model(input_shape=input_shape)
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

callbacks = [
    ModelCheckpoint(filepath='alexnet_best_synthetic_PCA.h5', verbose=1, save_best_only=True),
    ReduceLROnPlateau(factor=0.1, patience=10)
]

# Train the model
epochs = 30
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=(x_val, y_val), shuffle=True, callbacks=callbacks)

# Save the final model and training history
model.save('alexnet_final_model_synthetic_PCA.h5')
with open('alexnet_training_history_synthetic_PCA.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# Evaluate the model on the validation set
scores = model.evaluate(x_val, y_val, verbose=1)
y_preds = (model.predict(x_val) > 0.5).astype(int)
y_pred_binary = (y_preds > 0.5).astype(int)

f1 = f1_score(y_val.ravel(), y_preds.ravel(), average='macro')
tn, fp, fn, tp = confusion_matrix(y_val.ravel(), y_pred_binary.ravel()).ravel()
tpr1 = tp / (tp + fn)
fpr1 = fp / (fp + tn)
cm = confusion_matrix(y_val.ravel(), y_preds.ravel())

# Calculate ROC and Precision-Recall curves
fpr, tpr, _ = roc_curve(y_val.ravel(), y_preds.ravel())
roc_auc = auc(fpr, tpr)
precision, recall, _ = precision_recall_curve(y_val.ravel(), y_preds.ravel())
prc_auc = auc(recall, precision)

# Save metrics and curves
metrics = {
    'Validation Loss': scores[0],
    'Validation Accuracy': scores[1],
    'F1 Score': f1,
    'True Positive Rate' : tpr1,
    'False Positive Rate' : fpr1,
    'Confusion Matrix': cm.tolist(),
    'ROC AUC': roc_auc,
    'PRC AUC': prc_auc,
    'FPR Array': fpr.tolist(),
    'TPR Array': tpr.tolist(),
    'Precision Array': precision.tolist(),
    'Recall Array': recall.tolist()
}
with open('alexnet_evaluation_metrics_synthetic_PCA.json', 'w') as file:
    json.dump(metrics, file)

# Plot and save ROC Curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.savefig('alexnet_roc_curve_synthetic_PCA.png')

# Plot and save Precision-Recall Curve
plt.figure()
plt.plot(recall, precision, color='blue', lw=2, label='Precision-Recall curve (area = %0.2f)' % prc_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc="lower left")
plt.savefig('alexnet_precision_recall_curve_synthetic_PCA.png')
